In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd

TEST_SIZE = 0.2
RANDOM_STATE = 42

In [ ]:
from sklift.datasets import fetch_criteo

data, outcome, treatment = fetch_criteo(
    target_col="conversion",
    treatment_col="exposure",
    return_X_y_t=True,
    data_home="../data/",
    download_if_missing=True,
)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, t_train, t_test, y_train, y_test = train_test_split(
    data,
    treatment,
    outcome,
    test_size=TEST_SIZE,
    random_state=RANDOM_STATE,
    shuffle=True,
)
X_train.shape, X_test.shape, t_train.shape, t_test.shape, y_train.shape, y_test.shape

## S-learner

In [ ]:
from causalml.inference.meta import BaseSClassifier
from lightgbm import LGBMClassifier

model_s = BaseSClassifier(learner=LGBMClassifier())
model_s.fit(X_train, t_train, y_train)

In [ ]:
tau_test = model_s.predict(X_test)
tau_test.shape

In [ ]:
y_test.value_counts()

In [ ]:
eval_df = pd.DataFrame({"tau": tau_test.reshape(-1), "t": t_test, "y": y_test})

In [ ]:
eval_df

In [ ]:
from src.viz import plot_qini_curve

plot_qini_curve(
    eval_df["y"],
    eval_df["tau"],
    eval_df["t"],
)

In [ ]:
from src.viz import plot_uplift_bins

plot_uplift_bins(
    eval_df["t"],
    eval_df["y"],
    eval_df["tau"],
)

## T-learner

In [ ]:
from causalml.inference.meta import BaseTClassifier

model_t = BaseTClassifier(learner=LGBMClassifier())
model_t.fit(X_train, t_train, y_train)

In [ ]:
plot_qini_curve(
    y_test,
    model_t.predict(X_test).reshape(-1),
    t_test,
)

In [ ]:
plot_uplift_bins(
    t_test,
    y_test,
    model_t.predict(X_test).reshape(-1),
)